In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#error_index = 0
writer = SummaryWriter("./runs/FC512_submodel_5e-1_eval") #_BN
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)
print(device)

cuda


In [3]:
from importnb import Notebook

In [4]:
with Notebook(lazy=True):
    import F4F_model_Submodel
header = F4F_model_Submodel

In [5]:
# set randomness
seed = 0

header.set_randomness(seed) # ipynb module import means run all cell in file
# load dataset
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0821_to_fullmodels/"
batch_size = 32 # 32~ out of memory in 3080
num_train = 128000 #640000 #debug

train_dataloader,test_dataloader = header.get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

===INFO===
torch ver : 1.7.1
torchvision ver : 0.8.2 
GPU model : TITAN RTX
train dataset[4000], test dataset[1563] are loaded


In [6]:
dataiter = iter(train_dataloader)
images, labels = dataiter.next()

In [7]:
#In_layer_number = 34 # 34 conv5_1 convolution
#Out_layer_number = 36 # 36 conv5_1 relu 
error_index=0
max_epoch = 90
num_error = 64

In [8]:
"""
writer.flush()
f4f = header.F4F().to(device)
target_model = header.Target_model(vgg16_bn).to(device)
filter_set = target_model.get_layer(34).weight.data
filter_set = torch.reshape(filter_set,(512,512*3*3)).to(device)
error_info = header.make_error_info(error_index,num_error).to(device)
#writer.add_graph(target_model,(images,f4f,error_info))
writer.add_graph(f4f,(images,filter_set,error_info))
writer.close()
"""

'\nwriter.flush()\nf4f = header.F4F().to(device)\ntarget_model = header.Target_model(vgg16_bn).to(device)\nfilter_set = target_model.get_layer(34).weight.data\nfilter_set = torch.reshape(filter_set,(512,512*3*3)).to(device)\nerror_info = header.make_error_info(error_index,num_error).to(device)\n#writer.add_graph(target_model,(images,f4f,error_info))\nwriter.add_graph(f4f,(images,filter_set,error_info))\nwriter.close()\n'

In [9]:
log_file = "./log/3fc_64_acc_log_0906.txt"

In [10]:
f4f = header.F4F().to(device)
#optimizer = torch.optim.SGD(f4f.parameters,lr=1e-4,weight_decay=1e-4,momentum=0.9)
optimizer = torch.optim.SGD([i for i in f4f.parameters()],lr=5e-1,weight_decay=1e-4,momentum=0.9)
#loss_fn = nn.CrossEntropyLoss()
def loss_fn(loss1_ratio,pred,label, filter_orig,filter_f4f):
    if(loss1_ratio <0 or loss1_ratio >1 ):
        print("wrong parameter ratio ",loss1_ratio)
        return nan
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.MSELoss()
    a = loss1(pred,label)
    b = loss2(filter_orig,filter_f4f)
    return loss1_ratio * a + (1-loss1_ratio) * b


In [11]:
[i.size() for i in f4f.parameters()]

[torch.Size([4608, 4608]),
 torch.Size([4608]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([5120, 5120]),
 torch.Size([5120]),
 torch.Size([4608, 5120]),
 torch.Size([4608])]

In [12]:
if torch.cuda.device_count() >1 :
    print("data parallel start")
    f4f = nn.DataParallel(f4f).to(device)

In [13]:
#split_model = split_layer(vgg16_bn,0,Out_layer_number)

original_model = d_copy(vgg16_bn).to(device)
# subset of vgg16 (whole layer) with f4f
header.hook_register(vgg16_bn,num_error,error_index)
target_model = header.Target_model(vgg16_bn).to(device)



input  Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [14]:
#target_model.model

In [15]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
first_feature = []
first_label = []
offset_info = []
f = open(log_file,"w")
print("Submodel case")
f.close()

target_model = header.Target_model(vgg16_bn).to(device)
    
tmp= header.training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,batch_size,
                  log_file,writer,retrain_model_path,
                  loss_fn,optimizer,
                  num_error,max_epoch,True)
        # tmp : first_feature,first_label,offset_info
first_feature.append(tmp[0])
first_label.append(tmp[1])
offset_info.append(tmp[2])

Submodel case
..........
[1,   100] loss: 6.240892
..........
[1,   200] loss: 6.282614
..........
[1,   300] loss: 6.229099
..........
[1,   400] loss: 6.318948
..........
[1,   500] loss: 6.362757
..........
[1,   600] loss: 6.229998
..........
[1,   700] loss: 6.288514
..........
[1,   800] loss: 6.195211
..........
[1,   900] loss: 6.304827
..........
[1,  1000] loss: 6.328884
..........
[1,  1100] loss: 6.239037
..........
[1,  1200] loss: 6.302159
..........
[1,  1300] loss: 6.196006
..........
[1,  1400] loss: 6.277868
..........
[1,  1500] loss: 6.368196
..........
[1,  1600] loss: 6.238765
..........
[1,  1700] loss: 6.325353
..........
[1,  1800] loss: 6.180549
..........
[1,  1900] loss: 6.284350
..........
[1,  2000] loss: 6.320918
..........
[1,  2100] loss: 6.210711
..........
[1,  2200] loss: 6.276591
..........
[1,  2300] loss: 6.204494
..........
[1,  2400] loss: 6.244645
..........
[1,  2500] loss: 6.401408
..........
[1,  2600] loss: 6.273409
..........
[1,  2700] lo

In [16]:
writer.flush()
writer.close()

In [17]:
# 여기서부터는 feature 그림 보기 위한 것들입니다.
for i in range(10):
    print(first_feature[0][0][i],first_label[0][0][i].argmax())

tensor([-1.2949e+00, -7.4835e-01, -2.8660e+00, -1.3078e+00, -6.9675e-01,
        -1.1274e+00, -7.4309e-01, -1.4525e+00, -1.0095e+00,  8.0502e-01,
        -4.5142e-02, -1.9161e-02,  6.6789e-01,  8.0750e-01,  9.4017e-01,
        -2.7363e-01, -2.4230e-01,  4.4678e-01,  3.5910e-01,  1.4490e+00,
        -1.7707e+00, -1.0178e+00, -1.7609e+00,  3.6564e-01, -1.2447e+00,
        -4.6765e-01, -7.3489e-01,  1.1904e-01, -1.0987e-01,  3.9440e-01,
        -7.4150e-01,  4.6591e-01, -3.0082e-01, -4.9465e-01,  6.2079e-01,
        -1.4249e+00,  3.1941e-01, -7.3470e-01,  2.0024e-01, -2.3518e-01,
         3.5912e-01,  3.9649e-01,  1.0416e+00,  3.4948e-01,  2.9197e-01,
         8.6047e-01,  9.3698e-01,  4.3177e-01,  9.6084e-02, -1.5583e+00,
        -1.2716e+00, -2.3165e+00,  1.1884e+00,  1.5262e+00,  1.5222e+00,
         1.4590e-01,  6.0542e-01,  1.8448e-02,  3.0675e-01,  9.8754e-01,
         1.7141e+00,  1.5020e+00, -6.0426e-03,  2.1591e+00,  8.0917e-01,
         2.8540e-01,  1.3659e+00,  1.7023e+00,  9.6

In [18]:

w = 10
h = 10
cols = 32
rows = 16
def feature_print(pic):
    #print("test with 'after pooling 4 feature'")
    fig = plt.figure(figsize=(64,32))
    ax = []
    for i in range(cols*rows):
        ch = pic[i,:,:]
        ax.append(fig.add_subplot(rows,cols,i+1))
        ax[-1].set_title(str(i)+"th ch (14x14)")
        plt.imshow(ch)

In [19]:
feature_print(first_feature[0])

TypeError: list indices must be integers or slices, not tuple

<Figure size 4608x2304 with 0 Axes>